In [1]:
from pathlib import Path
from collections import defaultdict, Counter
import numpy as np
from conlleval import evaluate
from scipy.optimize import fmin_l_bfgs_b
ninf = -1e8

In [2]:
def read_data(path, column=0):
    """column=0 means input sequence, column=1 means label
    """
    with open(path) as f:
        lines = f.readlines()
    
    data = []
    sample = []
    
    for line in lines:
        formatted_line = line.strip()
        
        if len(formatted_line) > 0:
            split_data = formatted_line.split(" ")
            sample.append(split_data[column])

        else:
            data.append(sample)
            sample = []
            
    return data

In [3]:
full_dir = Path('full')
partial_dir = Path('partial')

x_data, y_data = read_data(partial_dir/'train', 0), read_data(partial_dir/'train', 1)

In [4]:
# number of instances in the dataset
len(x_data), len(y_data)

(700, 700)

In [5]:
# y labels
y_vocab = sorted(list(set([oo for o in y_data for oo in o]))); y_vocab

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-org',
 'I-per',
 'I-tim',
 'O']

In [6]:
# x vocab
x_vocab = list(set([oo for o in x_data for oo in o])); len(x_vocab)

4068

## Part I (i): Emission scores

In [7]:
def calc_e(x_data, y_data, x_vocab, y_vocab):
    count_emission = Counter([(x,y) for x_instance, y_instance in zip(x_data, y_data) for x, y in zip(x_instance, y_instance)])
    count_label = Counter([oo for o in y_data for oo in o])
    
    e_score = {}
    for y in y_vocab:
        for x in x_vocab:
            feature = f"emission:{y}+{x}"
            
            if (x,y) not in count_emission:
                e_score[feature] = ninf
            else:
                score = np.log(count_emission[(x,y)]  /  count_label[y])
                e_score[feature] = score
    
    return e_score

In [8]:
emission_dict = calc_e(x_data, y_data, x_vocab, y_vocab)

## Part I (ii): Transition scores

In [9]:
def calc_t(y_data, y_vocab):
    count_transition = Counter([ (y_prev, y) for y_instance in y_data for y_prev, y in zip(['START'] + y_instance, y_instance + ['STOP'])])
    count_label = Counter([y for y_instance in y_data for y in ['START'] + y_instance])
    
    f_score = {}
    for y_prev in ['START'] + y_vocab:
        for y in y_vocab + ['STOP']:
            feature = f"transition:{y_prev}+{y}"
            
            if (y_prev,y) not in count_transition:
                f_score[feature] = ninf
            else:
                score = np.log(count_transition[(y_prev,y)]  /  count_label[y_prev])
                f_score[feature] = score
    
    return f_score

In [10]:
transition_dict = calc_t(y_data, y_vocab)

In [11]:
feature_dict = {**transition_dict, **emission_dict}

## Part II (i): Compute Score

In [12]:
def compute_score(x_instance, y_instance, feature_dict):
    feature_count = defaultdict(int)
    
    for x, y in zip(x_instance, y_instance): feature_count[f"emission:{y}+{x}"] += 1
    
    for y_prev, y in zip(['START'] + y_instance, y_instance + ['STOP']):
        feature_count[f"transition:{y_prev}+{y}"] += 1
        
    score = sum([feature_dict[feat]*count for feat, count in feature_count.items()])
    return score

In [13]:
x_instance = "This is the second U.N.-Congolese offensive against militias in the region since the DRC 's constitutional referendum a week ago .".split()
y_instance = 'O O O O O O O O O O O O O B-geo O O O O O O O'.split()
compute_score(x_instance, y_instance, feature_dict)

-139.57175855522826

## Part II (ii): Viterbi decoding

In [14]:
def viterbi(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    scores = np.full( (n,d), ninf)
    bp = np.full( (n,d), 0, dtype=np.int)
    
    for i, y in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:START+{y}",  ninf)
        e_score = feature_dict.get( f"emission:{y}+{x_instance[0]}",  ninf)
        scores[0, i] = t_score + e_score
        
    for i in range(1, n):
        for y_i, y in enumerate(y_vocab):
            for y_prev_i, y_prev in enumerate(y_vocab):
                t_score = feature_dict.get( f"transition:{y_prev}+{y}", ninf)
                e_score = feature_dict.get( f"emission:{y}+{x_instance[i]}", ninf)
                score = t_score + e_score + scores[i-1, y_prev_i]
                if score > scores[i, y_i]:
                    scores[i, y_i] = score
                    bp[i, y_i] = y_prev_i
    
    final_score, final_bp = ninf, 0
    for i, y_prev in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:{y_prev}+STOP", ninf)
        score = t_score + scores[n-1, i]
        if score > final_score: 
            final_score = score
            final_bp = i
            
    decoded_sequence = [ y_vocab[final_bp], ]
    for i in range(n-1, 0, -1):
        final_bp = bp[i, final_bp]
        decoded_sequence = [ y_vocab[final_bp] ] + decoded_sequence
        
    return decoded_sequence

print(" ".join(viterbi(x_instance, y_vocab, feature_dict)),)
print(" ".join(y_instance))

O O O O O O O O O O O O O B-geo O O O O O O O
O O O O O O O O O O O O O B-geo O O O O O O O


In [ ]:
def inference(in_file_path, y_vocab, feature_dict, out_file_path):
    x_data = read_data(in_file_path, column=0)
    with open(out_file_path, 'w') as f:
        for x_instance in x_data:
            pred = viterbi(x_instance, y_vocab, feature_dict)
            for word, label in zip(x_instance, pred): f.write(f"{word} {label} \n")
            f.write('\n')

inference(partial_dir/'dev.in', y_vocab, feature_dict, partial_dir/'dev.p2.out')

## Part III (i): CRF loss
refer to [machine learning slides](https://drive.google.com/file/d/1RfPcnQigx4jdLtnTjjjI1Jgd2UufexhV/view?usp=sharing) for details

In [15]:
def logsumexp(a):
    b = a.max()
    return  b + np.log( (np.exp(a-b)).sum() )

def forward(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    scores = np.zeros( (n,d) )
    
    for i, y in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:START+{y}", ninf)
        scores[0, i] = t_score
    
    for i in range(1, n):
        for y_i, y in enumerate(y_vocab):
            temp = []
            for y_prev_i, y_prev in enumerate(y_vocab):
                t_score = feature_dict.get( f"transition:{y_prev}+{y}", ninf)
                e_score = feature_dict.get( f"emission:{y_prev}+{x_instance[i-1]}", ninf)
                temp.append(e_score + t_score + scores[i-1, y_prev_i])
            scores[i, y_i] = logsumexp(np.array(temp))
    
    temp = []
    for i, y_prev in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:{y_prev}+STOP", ninf)
        e_score = feature_dict.get( f"emission:{y_prev}+{x_instance[-1]}", ninf)
        temp.append(e_score + t_score + scores[-1, i])
    alpha = logsumexp(np.array(temp))
    
    return scores, alpha



def loss_fn_instance(x_instance, y_instance, feature_dict, y_vocab):
    first_term = compute_score(x_instance, y_instance, feature_dict)
    _, forward_score = forward(x_instance, y_vocab, feature_dict)
    return forward_score - first_term

In [16]:
loss_fn_instance(x_instance, y_instance, feature_dict, y_vocab)

1.3302877199753311

## Part III (ii): forward backward

In [17]:
def backward(x_instance, y_vocab, feature_dict, aggreg_fn=logsumexp):
    n, d = len(x_instance), len(y_vocab)
    scores = np.zeros( (n,d) )
    
    for i, y in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:{y}+STOP", ninf)
        e_score = feature_dict.get( f"emission:{y}+{x_instance[-1]}", ninf)
        scores[-1, i] = t_score + e_score
        
    for i in range(n-1, 0, -1):
        for y_i, y in enumerate(y_vocab):
            temp = []
            for y_next_i, y_next in enumerate(y_vocab):
                t_score = feature_dict.get( f"transition:{y}+{y_next}", ninf)
                e_score = feature_dict.get( f"emission:{y}+{x_instance[i-1]}")
                temp.append(e_score + t_score + scores[i, y_next_i])
            scores[i-1, y_i] = aggreg_fn(np.array(temp))
            
    temp = []
    for i, y_next in enumerate(y_vocab):
        t_score = feature_dict.get( f"transition:START+{y_next}")
        temp.append(t_score + scores[0, i])
    beta = aggreg_fn(np.array(temp))
    
    return scores, beta



def forward_backward(x_instance, y_vocab, feature_dict):
    n, d = len(x_instance), len(y_vocab)
    f_scores, alpha = forward(x_instance, y_vocab, feature_dict)
    b_scores, beta = backward(x_instance, y_vocab, feature_dict)
    
    feature_expected_count = defaultdict(float)
    
    for i in range(n):
        for y_i, y in enumerate(y_vocab):
            e_feature = f"emission:{y}+{x_instance[i]}"
            feature_expected_count[e_feature] += np.exp(f_scores[i, y_i] + b_scores[i, y_i] - alpha)
            
    for i, y_next in enumerate(y_vocab):
        t_feature = f"transition:START+{y_next}"
        feature_expected_count[t_feature] += np.exp(f_scores[0, i] + b_scores[0, i] - alpha)
        
        t_feature = f"transition:{y_next}+STOP"
        feature_expected_count[t_feature] += np.exp(f_scores[-1, i] + b_scores[-1, i] - alpha)
        
    for y_i, y in enumerate(y_vocab):
        for y_next_i, y_next in enumerate(y_vocab):
            t_feature = f"transition:{y}+{y_next}"
            t_score = feature_dict.get(t_feature, ninf)
            total = 0
            for i in range(n-1):
                e_score = feature_dict.get(f"emission:{y}+{x_instance[i]}", ninf)
                total += np.exp(f_scores[i, y_i] + b_scores[i+1, y_next_i] + t_score + e_score - alpha)
            feature_expected_count[t_feature] = total
            
    return feature_expected_count

def get_feature_count(x_instance, y_instance, feature_dict):
    feature_count = defaultdict(int)
    
    for x, y in zip(x_instance, y_instance): feature_count[f"emission:{y}+{x}"] += 1
    
    for y_prev, y in zip(['START'] + y_instance, y_instance + ['STOP']):
        feature_count[f"transition:{y_prev}+{y}"] += 1
    
    return feature_count


    

## Part IV (i): gradient and training with regularization

In [18]:
def gradient_fn(x_data, y_data, feature_dict, y_vocab, eta=0):
    feature_grad = defaultdict(float)
    
    for x_instance, y_instance in zip(x_data, y_data):
        feature_expected_counts = forward_backward(x_instance, y_vocab, feature_dict)
        feature_actual_counts = get_feature_count(x_instance, y_instance, feature_dict)
        for k, v in feature_expected_counts.items(): feature_grad[k] += v
        for k, v in feature_actual_counts.items(): feature_grad[k] -= v    
    
    if eta > 0: 
        for k, v in feature_dict.items(): feature_grad[k] += 2*eta*v
    
    return feature_grad
        
    
def loss_fn(x_data, y_data, feature_dict, y_vocab, eta=0):
    loss = 0
    for x_instance, y_instance in zip(x_data, y_data):
        loss += loss_fn_instance(x_instance, y_instance, feature_dict, y_vocab) 
    reg_loss = eta * sum([o**2 for o in feature_dict.values()]) if eta > 0 else 0
    return loss + reg_loss

In [19]:
# Gradient verification
feature_key_checks = ['emission:O+the', 'transition:START+O', 'transition:O+O', 'transition:I-per+I-per']
feature_gradients = gradient_fn(x_data, y_data, feature_dict, y_vocab, eta=0)
loss1 = loss_fn(x_data, y_data, feature_dict, y_vocab, eta=0)
delta = 1e-6
for feat_k in feature_key_checks:
    new_feature_dict = feature_dict.copy()
    new_feature_dict[feat_k] += delta
    loss2 = loss_fn(x_data, y_data, new_feature_dict, y_vocab, eta=0)
    numerical_grad = (loss2 - loss1) / delta
    analytic_grad = feature_gradients[feat_k]
    assert abs(numerical_grad - analytic_grad) / max(abs(numerical_grad), 1e-8) < 1e-5, 'Did not pass gradient checking'

In [23]:
# Helper function
def numpy_to_dict(weight, feature_dict):
    for i,k in enumerate(feature_dict.keys()):
        feature_dict[k] = weight[i]
    return feature_dict

def dict_to_numpy(grads, feature_dict):
    np_grads = np.zeros(len(feature_dict))
    for i, k in enumerate(feature_dict.keys()):
        np_grads[i] = grads[k]
    return np_grads

def get_loss_grad(weight, *args):
    x_data, y_data, feature_dict, y_vocab = args
    feature_dict = numpy_to_dict(weight, feature_dict)
    loss = loss_fn(x_data, y_data, feature_dict, y_vocab, eta=0.1)
    grads = gradient_fn(x_data, y_data, feature_dict, y_vocab, eta=0.1)
    grads = dict_to_numpy(grads, feature_dict)
    return loss, grads

def callbackF(weight): print(f'Loss: \t {loss_fn(x_data, y_data, feature_dict, y_vocab, eta=0.1):.4f}')

In [ ]:
init_weight = np.random.random(len(feature_dict))
feature_dict = numpy_to_dict(init_weight, feature_dict)
result = fmin_l_bfgs_b(get_loss_grad, 
                       init_weight, 
                       args=(x_data, y_data, feature_dict, y_vocab), 
                       pgtol=0.01, 
                       callback=callbackF)

44478.69569966906
33525.67208654845
20308.133944778292
Loss: 	 20308.133944778292
16107.921421833915
Loss: 	 16107.921421833915
41150.37349280199
15146.290018266269
Loss: 	 15146.290018266269
14921.717489489878
Loss: 	 14921.717489489878
14463.920914536946
Loss: 	 14463.920914536946
14017.996594974013
Loss: 	 14017.996594974013
12839.139916211498
Loss: 	 12839.139916211498
11588.446676907115
Loss: 	 11588.446676907115
11474.069639827982
10987.306013525267
Loss: 	 10987.306013525267
10619.654838586039
Loss: 	 10619.654838586039
10179.653289703536
Loss: 	 10179.653289703536
9788.483570045282
Loss: 	 9788.483570045282
9462.625389117944
Loss: 	 9462.625389117944
9200.22891879007
Loss: 	 9200.22891879007
8946.50935893688
Loss: 	 8946.50935893688
8585.657052324774
Loss: 	 8585.657052324774
8256.336089962046
Loss: 	 8256.336089962046
8075.0346050803155
Loss: 	 8075.0346050803155
7961.788518610383
Loss: 	 7961.788518610383
7757.598193097727
Loss: 	 7757.598193097727
7480.577805764386
Loss: 	 7

## Part 4 (ii): write to file

In [ ]:
inferece(partial_dir/'dev.in', y_vocab, feature_dict, partial_dir/'dev.p4.out')

## Part 5 (i)

## Part 5 (ii)

## Part 5 (iii) Structured Perceptron

In [ ]:
logsumexp = max